In [72]:
import pandas as pd
import numpy as np

In [73]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values
print(X[0])

[619 'France' 'Female' 42 2 0.0 1 1 1.0 101348.88]


In [74]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1.0,101348.88,1.0
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1.0,112542.58,0.0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0.0,113931.57,1.0
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0.0,93826.63,0.0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1.0,79084.10,0.0


In [75]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X[0])

[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1.0 101348.88]


In [76]:
print(X[0])

[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1.0 101348.88]


In [77]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(X_train[0])
print(type(X_train))

[1.0 0.0 0.0 778 1 33 1 0.0 2 1 0.0 85439.73]
<class 'numpy.ndarray'>


In [78]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [87]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Activation, Dropout
from keras.activations import relu, sigmoid
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.models import Sequential

def create_model(layers, activation):
    model = Sequential()
    for i,nodes in enumerate(layers):
        if i==0: # For 1st layer, we need to provide input dimensions
            model.add(Dense(nodes, input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3)) # Dropout will deactive few neuron to avoid overfitting problem
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3)) # Dropout will deactive few neuron to avoid overfitting problem
        
    model.add(Dense(units=1, activation=sigmoid, kernel_initializer=glorot_uniform)) # o/p layer
    model.compile(optimizer='adam', loss=binary_crossentropy, metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [88]:
model

In [90]:
layers = [(20,),(40,20),(45,30,15)] # [20]-1 layer with 20 neurons, [40,20]-2 layers(1st-40, 2nd-20) etc
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size=[128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_result=grid.fit(X_train,y_train)

In [ ]:
print(grid_result.best_score_,grid_result.best_params_)

y_pred = grid.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

score = accuracy_score(y_test, y_pred)
print('score : ', score)